In [19]:
import os
import json
import numpy as np
import ultralytics
from ultralytics import YOLO
import cv2

In [20]:
models = {
    "no_aug_300_nano": YOLO("E:/Project TA/Caroline/runs/segment/train17/weights/best.pt"),
    "no_aug_300_small": YOLO("E:/Project TA/Caroline/runs/segment/train24/weights/best.pt"),
    "aug_300_nano":    YOLO("E:/Project TA/Caroline/runs/segment/train25/weights/best.pt"),
    "aug_300_small":    YOLO("E:/Project TA/Caroline/runs/segment/train26/weights/best.pt"),
}

In [21]:
# Path direktori
images_dir = "E:/Project TA/iou/Testing IoU/test/images"
labels_dir = "E:/Project TA/iou/Testing IoU/test/labels"
output_dir = "E:/Project TA/iou/predict_iou/AdamW" 

In [22]:
os.makedirs(output_dir, exist_ok=True)

In [23]:
# Fungsi IoU
def calculate_pairwise_iou_with_overlap_check(ground_truth, predicted, image_size):
    pairwise_ious = []
    for gt_polygon in ground_truth:
        for pred_polygon in predicted:
            gt_mask = np.zeros(image_size, dtype=np.uint8)
            pred_mask = np.zeros(image_size, dtype=np.uint8)
            cv2.fillPoly(gt_mask, [gt_polygon], 1)
            cv2.fillPoly(pred_mask, [pred_polygon], 1)
            intersection = np.logical_and(gt_mask, pred_mask).sum()
            if intersection > 0:
                union = np.logical_or(gt_mask, pred_mask).sum()
                iou = intersection / union if union > 0 else 0
                pairwise_ious.append((gt_polygon, pred_polygon, iou))
    return pairwise_ious

In [24]:
for model_name, model in models.items():
    print(f"\nMemproses model: {model_name}")
    save_dir = os.path.join(output_dir, model_name)
    os.makedirs(save_dir, exist_ok=True)

    for image_file in os.listdir(images_dir):
        if not image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        image_path = os.path.join(images_dir, image_file)
        label_path = os.path.join(labels_dir, os.path.splitext(image_file)[0] + ".txt")

        frame = cv2.imread(image_path)
        if frame is None:
            print(f"Gagal baca gambar: {image_path}")
            continue

        height, width, _ = frame.shape
        frame_copy = frame.copy()

        # === Ground Truth ===
        ground_truth = []
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f.readlines():
                    values = list(map(float, line.split()))
                    coords = np.array(values[1:]).reshape(-1, 2)
                    coords[:, 0] *= width
                    coords[:, 1] *= height
                    ground_truth.append(coords.astype(int))
        else:
            continue

        # === Predict ===
        results = model.predict(frame, imgsz=640, conf=0.3, verbose=False)
        result = results[0]

        masks = result.masks.xy if result.masks is not None else []
        predicted = [np.array(mask).astype(int) for mask in masks]

        # === Hitung IoU ===
        pairwise_ious = calculate_pairwise_iou_with_overlap_check(ground_truth, predicted, (height, width))

        # === Deteksi FP dan FN ===
        matched_gt_indices = set()
        matched_pred_indices = set()

        for gt, pred, _ in pairwise_ious:
            # Cari index ground truth
            gt_idx = next((i for i, g in enumerate(ground_truth) if np.array_equal(g, gt)), None)
            # Cari index prediction
            pred_idx = next((i for i, p in enumerate(predicted) if np.array_equal(p, pred)), None)

            if gt_idx is not None:
                matched_gt_indices.add(gt_idx)
            if pred_idx is not None:
                matched_pred_indices.add(pred_idx)


        fn_indices = [i for i in range(len(ground_truth)) if i not in matched_gt_indices]
        fp_indices = [i for i in range(len(predicted)) if i not in matched_pred_indices]

        # === Gambar semua ground truth (hijau) ===
        for gt in ground_truth:
            cv2.polylines(frame_copy, [gt], isClosed=True, color=(0, 255, 0), thickness=2)

        # === Gambar semua prediksi (merah) ===
        for pred in predicted:
            cv2.polylines(frame_copy, [pred], isClosed=True, color=(0, 0, 255), thickness=2)

        # === Tambah teks IoU hanya untuk pasangan TP ===
        for gt, pred, iou in pairwise_ious:
            center = np.mean(gt, axis=0).astype(int)
            text = f"IoU: {iou:.2f}"
            (font_w, font_h), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
            x, y = center
            cv2.rectangle(frame_copy, (x, y - font_h - 4), (x + font_w + 4, y + 4), (255, 255, 255), -1)
            cv2.putText(frame_copy, text, (x + 2, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

        # === Evaluasi metrik ===
        TP = len(pairwise_ious)
        FP = len(fp_indices)
        FN = len(fn_indices)

        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

        print(f"Gambar: {image_file} | TP: {TP}, FP: {FP}, FN: {FN} | Precision: {precision:.2f}, Recall: {recall:.2f}, F1: {f1:.2f}")

        # Simpan gambar hasil
        save_path = os.path.join(save_dir, f"{os.path.splitext(image_file)[0]}_iou.jpg")
        cv2.imwrite(save_path, frame_copy)

    print(f"Hasil disimpan di folder: {save_dir}")


Memproses model: no_aug_300_nano
Gambar: banjir_jpg.rf.950c421f52bc7c06dfe21e7ad33117ae.jpg | TP: 0, FP: 1, FN: 0 | Precision: 0.00, Recall: 0.00, F1: 0.00
Gambar: genangan_rumput_jpg.rf.2a5ba86b223a8c9cce6eca7b8c3345ae.jpg | TP: 1, FP: 0, FN: 0 | Precision: 1.00, Recall: 1.00, F1: 1.00
Gambar: jalan_batu_jpeg.rf.5eb09403644b6701b2026399b6b50508.jpg | TP: 10, FP: 0, FN: 2 | Precision: 1.00, Recall: 0.83, F1: 0.91
Gambar: lubang_genangan_jpg.rf.9f235615ec131e8c3375b0b755dd56d8.jpg | TP: 1, FP: 0, FN: 0 | Precision: 1.00, Recall: 1.00, F1: 1.00
Gambar: lubang_jpeg.rf.7b4f2c57e411fb82b55c670f4784c838.jpg | TP: 0, FP: 0, FN: 0 | Precision: 0.00, Recall: 0.00, F1: 0.00
Hasil disimpan di folder: E:/Project TA/iou/predict_iou/AdamW\no_aug_300_nano

Memproses model: no_aug_300_small
Gambar: banjir_jpg.rf.950c421f52bc7c06dfe21e7ad33117ae.jpg | TP: 0, FP: 1, FN: 0 | Precision: 0.00, Recall: 0.00, F1: 0.00
Gambar: genangan_rumput_jpg.rf.2a5ba86b223a8c9cce6eca7b8c3345ae.jpg | TP: 1, FP: 0, FN: 0 